In [1]:
!pip install selenium

import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt



In [7]:
comment_data = pd.read_csv('문재인.csv')

In [8]:
comment_data

,Video ID,Title,Comment
0,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",마지막 통화 내용이나\r\n말해주세요
1,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",신 대표님 마스크는 추천하지 마세요! 기능이 떨어지는 저급을 많은돈 지급하고 사게...
2,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",여가부가무엇하는곳인지알고싶어요
3,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",아마죽었다고쏘하는지도몰라요 시신도보여주지도안고 죽었다고만하니못믿어워요 그미친년놈들 ...
4,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",사인이.무었인가도모르고초상친나.
...,...,...,...
6086,K4zzx2yljNM,(다반뉴스) 문재인 감빵가자! 조국사건 대통령 개입 터졌다! / 신의한수 20.06.25,문제덩어리 빵간다니 속이 시원합니다
6087,K4zzx2yljNM,(다반뉴스) 문재인 감빵가자! 조국사건 대통령 개입 터졌다! / 신의한수 20.06.25,그런데 판사 세끼들이 원칙과 양심에 따라재판\r\n해야 하는데 그렇치 못하니 ...
6088,K4zzx2yljNM,(다반뉴스) 문재인 감빵가자! 조국사건 대통령 개입 터졌다! / 신의한수 20.06.25,한마디로 문조현 토착간첩 정권답네요~갈수록 태산이다.\r\n조현병 환자를 반대하는 ...
6089,K4zzx2yljNM,(다반뉴스) 문재인 감빵가자! 조국사건 대통령 개입 터졌다! / 신의한수 20.06.25,신의한수 식구들~♡♡힘내세요\r\n화이팅🤗🤗👍👍👏👏👏💖💖💖💖


In [9]:

#이모티콘 제거
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

In [10]:
comment_list = []
for i in range(len(comment_data)):
    comment_list.append(comment_data['Comment'].iloc[i])

In [11]:
comment_result = []

for i in comment_list:
    tokens = re.sub(emoji_pattern,"",i)
    tokens = re.sub(han,"",tokens)
    comment_result.append(tokens)

comment_result = pd.DataFrame(comment_result, columns=["Comment"])

In [12]:
comment_result

,Comment
0,마지막 통화 내용이나말해주세요
1,신 대표님 마스크는 추천하지 마세요 기능이 떨어지는 저급을 많은돈 지급하고 사게되...
2,여가부가무엇하는곳인지알고싶어요
3,아마죽었다고쏘하는지도몰라요 시신도보여주지도안고 죽었다고만하니못믿어워요 그미친년놈들 ...
4,사인이무었인가도모르고초상친나
...,...
6086,문제덩어리 빵간다니 속이 시원합니다
6087,그런데 판사 세끼들이 원칙과 양심에 따라재판해야 하는데 그렇치 못하니 답답하네요
6088,한마디로 문조현 토착간첩 정권답네요갈수록 태산이다조현병 환자를 반대하는 국민은 살수...
6089,신의한수 식구들♡♡힘내세요화이팅🤗🤗


## 형태소 분석기를 통한 명사 추출¶

In [13]:
# !pip install konlpy

In [14]:
#한글 형태소 분석기인 konlpy사용 
#이때 konlpy에는 hannanum, kommoran, kkma, twitter이 있는데 해당 분석에서는 kkma사용
from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.tag import Twitter
#apply lambda사용

In [15]:
from konlpy.tag import Twitter

def get_noun(comment_txt):
    twitter = Twitter()
    noun = []
    
    if len(comment_txt)>0:
        tw = twitter.pos(comment_txt)
        for i,j in tw:
            if j == 'Noun':
                noun.append(i)
    return noun
 
comment_result['token'] = comment_result['Comment'].apply(lambda x: get_noun(x))

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [16]:
comment_result['token']

0                                        [마지막, 통화, 내용, 말]
1       [신, 대표, 마스크, 추천, 기능, 저급, 돈, 지급, 면, 신의, 한수, 정도,...
2                                            [여가부, 무엇, 곳]
3                          [아마, 시신, 도안, 워, 그, 년놈, 죽, 좃겠어]
4                                     [사인, 무었, 가도, 초상, 친]
                              ...                        
6086                                     [문제, 덩어리, 빵, 간다]
6087                                  [판사, 끼, 원칙, 양심, 재판]
6088    [한마디, 조현, 토착, 간첩, 정권, 답, 갈수록, 태산, 조현병, 환자, 반대,...
6089                                      [신의한수, 식구, 화이팅]
6090                                                 [과연]
Name: token, Length: 6091, dtype: object